# Small tutorial on iRODS paths

* iRODS deals with POSIX-like paths, i.e. paths use the `/` as delimiter and all absolute paths start with `/`.
* In default iRODS instances users have a personal home collection `/<zonename>/home/<username>`
* In some iRODS instances like in Yoda instances, users are part of a group and only have access to group collections `/<yodazone>/home/<groupname>`

In *iBridges* we allow all paths by default to be strings, which have the to be formatted as described above, i.e. for up and downloads all source and destination paths can be offered as `str`.
However, we also offer a small class `IrodsPath` with which some formatting is done automatically.

## The iRODS home

In your configuration file `irods_environment.json` you can set a default working directory on the iRODS instance with:
```
"irods_home": "/<zone>/home/<user or groupname>"
```
This information is stored in the iBridges session object and employed by the iBridges IrodsPath object.

### Create a session

In [ ]:
from ibridges import Session
import os, json
from getpass import getpass

In [ ]:
with open(os.path.expanduser("~/.irods/irods_environment.json"), "r") as f:
    ienv = json.load(f)
password = getpass("Your iRODS password")
session = Session(irods_env=ienv, password=password, irods_home='')

In [ ]:
print(session.home())

If you did not set an "irods_home" in the configuration file, the `session` will set it automatically to `/<zonename>/home/<username>`. You can also overwrite your setting in the configuration file by setting a vlue to `irods_home` when you create the `session`.

**Note**, that it is not verified that this path really exists on the iRODS server. It is merely a configuration item.

### Verify that your current working directory exists

In [ ]:
from ibridges import IrodsPath
home_path = IrodsPath(session, session.home())
print(home_path)
print(home_path.exists())
print(home_path.collection_exists())

###  Create an IrodsPath

In [ ]:
ipath = IrodsPath(session, 'blabla', 'blibli')

Let's expect the properties and functions of the empty iRODS path.

In [ ]:
print(ipath)
print(ipath.absolute_path())
print(ipath.parts)
print(ipath.exists())

When we create a new iRODS path which does not start with `/` the `IrodsPath` will automatically expand the path with the current working directory, i.e the iRODS home.

### Extending an iRODS path

Asssume we would like to create a new path in iRODS e.g. to upload data to or to create a new collection. We want to extend our `home_path` with `testdata/experiment1`.

In [ ]:
new_loc = home_path.joinpath('testdata', 'experiment1')
print(new_loc)

### Joining iRODS and Windows paths
Assume you are working on windows and would like to create a copy of a local working folder on iRODS. Windows uses a different parts separator `\`. In such a case we advise you to work with the `pathlib.Path` or the `pathlib.WindowsPath`.

In [ ]:
from pathlib import PureWindowsPath
win_path = PureWindowsPath('winodws','path','to', 'study', 'experiment2')
print(win_path)

We can assemble the new iRODS path `study/experiment2` like this:

In [ ]:
print(win_path.parts[-2:])
new_loc = home_path.joinpath(win_path.parts[-2], win_path.parts[-1])
print(new_loc)

Similarly we can extend a Windows path with a relative iRODS path: 

In [ ]:
print(ipath)
new_loc = win_path.joinpath(ipath.parts[0], ipath.parts[1])
print(new_loc)